In [1]:
import jax
from neurallogic import hard_not, hard_and, harden, neural_logic_net

In [3]:
x = [True, False]
w = [[True, True], [False, True], [True, False], [False, False]]
[[hard_and.symbolic_and_include(wj, xi) for wj, xi in zip(wi, x)] for wi in w]

TypeError: bad operand type for unary ~: 'list'